## A DATASET for GPS Spoofing Detection on Unmanned Aerial System

In [27]:
# Pacotes básicos
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Pacotes do Scikit-learn para acesso a datasets, preparação, modelagem e avaliação
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Pacotes para modelos de aprendizado
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.tree import DecisionTreeClassifier

# Pacote de balanceamento de classes desbalanceadas
from imblearn.over_sampling import SMOTE

In [28]:
# df = pd.read_csv("GPS_Data_Simplified_2D_Feature_Map.csv")
df = pd.read_csv("GPS_Data_Simplified_2D_Feature_Map.csv", delimiter=";")

for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = pd.to_numeric(df[col].str.replace(",", ".", regex=False), errors="coerce")

print(df.dtypes)
df.head()

PRN         int64
DO        float64
PD        float64
RX        float64
TOW       float64
CP        float64
EC        float64
LC        float64
PC        float64
PIP       float64
PQP       float64
TCD       float64
CN0       float64
Output      int64
dtype: object


,PRN,DO,PD,RX,TOW,CP,EC,LC,PC,PIP,PQP,TCD,CN0,Output
0,6,1160.671408,-238359.5433,491568.00,491568.0008,-24660.26329,163521.78,170008.81,179294.97,178420.33,-17688.17800,1158.4806,49.412529,0
1,6,1157.827808,-238363.9212,491568.02,491568.0208,-24683.46852,158277.05,160253.09,182106.11,182106.05,158.33333,1160.7948,49.452686,0
2,6,1161.135932,-238368.6386,491568.04,491568.0408,-24706.68736,184442.36,193125.89,208228.02,-205198.30,35391.59000,1157.6909,49.480145,0
3,6,1161.156617,-238373.3918,491568.06,491568.0608,-24729.92004,159812.45,169960.55,187550.72,187543.00,-1701.94400,1161.1343,49.433769,0
4,6,1160.690893,-238378.1271,491568.08,491568.0808,-24753.15400,183557.66,191912.58,208403.50,208330.05,-5532.89990,1161.1759,49.391670,0


In [29]:
print(df.shape)

(510530, 14)


#### Features
- **Satellite Vehicle Number (PRN)** - Receiver stage: Post-correlation. uid for each gps constallation satellite.

- **Carrier Doppler in Hz (DO)** - Receiver stage: Post-correlation. The Doppler shift is the result of the satellite and receiver motion. The DO is expressed as the frequency drift between the sent frequency and received frequency of the GPS signal.

- **Pseudo-range in meter (PD)** - Receiver stage: Post-correlation. The pseudo-range is the time difference between the transmission and reception time. It can be expressed in meters which refer to the distance between the receiver and the satellites.
- **Time of the Week in seconds (TOW)** - Receiver stage: Post-correlation. It refers to the number of elapsed seconds since the start of the week given by the satellite atomic clock. When TOW reaches to the end of the week which is 604,799 seconds it resets to zero.

- **Receiver Time (RX)** - Receiver stage: Post-correlation. It is the receiver time given in seconds after the start of the TOW.

- **Carrier Phase Cycles (CP)** - Receiver stage: Post-correlation. It refers to the beat frequency difference between the received carrier of the satellite signal and a receiver - generated reference frequency given in cycles.

**Magnitude of the Early Correlator (EC)**      During correlation
**Magnitude of the Late Correlator (LC)**       During correlation
**The Magnitude of the Prompt Correlator (PC)** During correlation
**Prompt in phase correlator (PIP)**            During correlation
**Prompt Quadrature Component (PQP)**           During correlation
**Carrier Doppler in Tracking loop in Hz(TCD)** During correlation
**Carrier to Noise Ratio in dB-Hz (C/N◦)**      Pre-correlation

In [30]:
df.describe()

,PRN,DO,PD,RX,TOW,CP,EC,LC,PC,PIP,PQP,TCD,CN0,Output
count,510530.000000,510530.000000,5.105300e+05,510530.000000,510530.000000,5.105300e+05,510530.000000,510530.000000,510530.000000,510530.000000,510530.000000,510530.000000,510530.000000,510530.000000
mean,14.399743,1293.270723,7.992497e+06,282666.233354,282666.208111,-1.216893e+05,107973.452869,107946.509101,119563.129474,39.357453,65.535775,1303.725859,44.717167,0.432819
std,9.279444,2362.534207,1.006911e+07,106925.234281,106925.232122,2.398627e+05,31711.904090,31572.262598,34068.953790,122138.198997,24318.316229,2348.382188,2.909404,0.891641
min,2.000000,-4262.754835,-6.588428e+06,173640.000000,173640.008000,-1.032332e+06,-11261.148740,-12557.513790,318.674133,-270777.625000,-216012.859900,-3214.671400,0.000000,0.000000
25%,7.000000,-868.884393,2.374256e+06,174220.600000,174220.522000,-2.560281e+05,86407.752045,86486.361330,96580.286923,-119618.003925,-11904.076640,-864.922714,43.203264,0.000000
50%,11.000000,1425.205130,3.777471e+06,263290.160000,263290.145050,-9.741575e+04,109815.675800,109799.156250,121863.750000,-339.778535,35.455368,1472.187866,45.346006,0.000000
75%,26.000000,3451.204770,6.814777e+06,263799.780000,263799.770600,6.364966e+04,130963.429675,130794.406300,144362.848125,119631.968800,11955.028080,3444.271612,46.691953,0.000000
max,31.000000,6715.759846,2.830762e+07,492039.420000,492039.341100,3.527992e+05,244872.734400,244650.390600,271466.125000,242361.078100,226761.110000,5260.343750,50.482834,3.000000
